### Instalando a biblioteca Pyspark e Findspark

In [21]:
#!pip install pyspark

In [22]:
g#!pip install findspark

### Importando Pyspark - iniciando FindSpark

In [23]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [24]:
df = spark.sql("""select 'sucesso total estamos online!' as hello""")
df.show()

+--------------------+
|               hello|
+--------------------+
|sucesso total est...|
+--------------------+



### Importando funções especitficas para tratamento de dados

In [25]:
from pyspark.sql import Row, DataFrame
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import col, expr, lit, substring, concat, concat_ws, when, coalesce
from pyspark.sql import functions as F
from functools import reduce

In [26]:
from os import sep

df = spark.read.csv('banklist.csv', sep= ',', inferSchema=True, header=True)

print('df.count', df.count())
print('df.columns', df.columns)
print("df.lens", len(df.columns))

df.count 561
df.columns ['Bank Name', 'City', 'ST', 'CERT', 'Acquiring Institution', 'Closing Date']
df.lens 6


### Usando SQL no Spark

In [27]:
df.createOrReplaceTempView('banklist')
df_check = spark.sql('''select `Bank Name`, City, `Closing Date` from banklist''')
df_check.show(10)

+--------------------+-------------+------------+
|           Bank Name|         City|Closing Date|
+--------------------+-------------+------------+
|The First State Bank|Barboursville|    3-Apr-20|
|  Ericson State Bank|      Ericson|   14-Feb-20|
|City National Ban...|       Newark|    1-Nov-19|
|       Resolute Bank|       Maumee|   25-Oct-19|
|Louisa Community ...|       Louisa|   25-Oct-19|
|The Enloe State Bank|       Cooper|   31-May-19|
|Washington Federa...|      Chicago|   15-Dec-17|
|The Farmers and M...|      Argonia|   13-Oct-17|
| Fayette County Bank|   Saint Elmo|   26-May-17|
|Guaranty Bank, (d...|    Milwaukee|    5-May-17|
+--------------------+-------------+------------+
only showing top 10 rows



In [28]:
df.describe().show()

+-------+--------------------+-------+----+-----------------+---------------------+------------+
|summary|           Bank Name|   City|  ST|             CERT|Acquiring Institution|Closing Date|
+-------+--------------------+-------+----+-----------------+---------------------+------------+
|  count|                 561|    561| 561|              561|                  561|         561|
|   mean|                NULL|   NULL|NULL|31685.68449197861|                 NULL|        NULL|
| stddev|                NULL|   NULL|NULL|16446.65659309965|                 NULL|        NULL|
|    min|1st American Stat...|Acworth|  AL|               91|      1st United Bank|    1-Aug-08|
|    max|               ebank|Wyoming|  WY|            58701|  Your Community Bank|    9-Sep-11|
+-------+--------------------+-------+----+-----------------+---------------------+------------+



In [29]:
df.describe('City', 'ST').show()

+-------+-------+----+
|summary|   City|  ST|
+-------+-------+----+
|  count|    561| 561|
|   mean|   NULL|NULL|
| stddev|   NULL|NULL|
|    min|Acworth|  AL|
|    max|Wyoming|  WY|
+-------+-------+----+



### Usando funções Count , colunms e Schema

In [30]:
print('Total de linhas:' , df.count())
print('Total de colunas:', len(df.columns))
print('colunas:', df.columns)
print('Tipos de dados:', df.dtypes)
print()
print('Schema:', df.printSchema())

Total de linhas: 561
Total de colunas: 6
colunas: ['Bank Name', 'City', 'ST', 'CERT', 'Acquiring Institution', 'Closing Date']
Tipos de dados: [('Bank Name', 'string'), ('City', 'string'), ('ST', 'string'), ('CERT', 'int'), ('Acquiring Institution', 'string'), ('Closing Date', 'string')]

root
 |-- Bank Name: string (nullable = true)
 |-- City: string (nullable = true)
 |-- ST: string (nullable = true)
 |-- CERT: integer (nullable = true)
 |-- Acquiring Institution: string (nullable = true)
 |-- Closing Date: string (nullable = true)

Schema: None


In [31]:
df.dropDuplicates()
print('Total de linhas:', df.count())
print('Total de colunas:', len(df.columns))

Total de linhas: 561
Total de colunas: 6


In [32]:
df2 = df.select('Bank Name', 'City')
df2.show(5)

+--------------------+-------------+
|           Bank Name|         City|
+--------------------+-------------+
|The First State Bank|Barboursville|
|  Ericson State Bank|      Ericson|
|City National Ban...|       Newark|
|       Resolute Bank|       Maumee|
|Louisa Community ...|       Louisa|
+--------------------+-------------+
only showing top 5 rows



### Removendo colunas

In [33]:
col_1 = list(set(df.columns) - {'CERT','ST'})
df2 = df.select(col_1)
df2.show(5)

+--------------------+---------------------+-------------+------------+
|           Bank Name|Acquiring Institution|         City|Closing Date|
+--------------------+---------------------+-------------+------------+
|The First State Bank|       MVB Bank, Inc.|Barboursville|    3-Apr-20|
|  Ericson State Bank| Farmers and Merch...|      Ericson|   14-Feb-20|
|City National Ban...|      Industrial Bank|       Newark|    1-Nov-19|
|       Resolute Bank|   Buckeye State Bank|       Maumee|   25-Oct-19|
|Louisa Community ...| Kentucky Farmers ...|       Louisa|   25-Oct-19|
+--------------------+---------------------+-------------+------------+
only showing top 5 rows



### Renomeando Colunas

In [34]:
df2 = df \
  .withColumnRenamed('Bank Name', 'nome_do_Banco') \
  .withColumnRenamed('City', 'cidade') \
  .withColumnRenamed('ST', 'estado') \
  .withColumnRenamed('Address', 'endereço') \
  .withColumnRenamed('Closing Date', 'data_de_fechamento') \
  .withColumnRenamed('Acquiring Institution', 'instituição_de_acquirência') \
  .withColumnRenamed('CERT', 'cert')

df2.show(5)

+--------------------+-------------+------+-----+--------------------------+------------------+
|       nome_do_Banco|       cidade|estado| cert|instituição_de_acquirência|data_de_fechamento|
+--------------------+-------------+------+-----+--------------------------+------------------+
|The First State Bank|Barboursville|    WV|14361|            MVB Bank, Inc.|          3-Apr-20|
|  Ericson State Bank|      Ericson|    NE|18265|      Farmers and Merch...|         14-Feb-20|
|City National Ban...|       Newark|    NJ|21111|           Industrial Bank|          1-Nov-19|
|       Resolute Bank|       Maumee|    OH|58317|        Buckeye State Bank|         25-Oct-19|
|Louisa Community ...|       Louisa|    KY|58112|      Kentucky Farmers ...|         25-Oct-19|
+--------------------+-------------+------+-----+--------------------------+------------------+
only showing top 5 rows




### Adiciona Colunas

In [35]:
df2 = df.withColumn('state', col('ST'))
df2.show(5)

+--------------------+-------------+---+-----+---------------------+------------+-----+
|           Bank Name|         City| ST| CERT|Acquiring Institution|Closing Date|state|
+--------------------+-------------+---+-----+---------------------+------------+-----+
|The First State Bank|Barboursville| WV|14361|       MVB Bank, Inc.|    3-Apr-20|   WV|
|  Ericson State Bank|      Ericson| NE|18265| Farmers and Merch...|   14-Feb-20|   NE|
|City National Ban...|       Newark| NJ|21111|      Industrial Bank|    1-Nov-19|   NJ|
|       Resolute Bank|       Maumee| OH|58317|   Buckeye State Bank|   25-Oct-19|   OH|
|Louisa Community ...|       Louisa| KY|58112| Kentucky Farmers ...|   25-Oct-19|   KY|
+--------------------+-------------+---+-----+---------------------+------------+-----+
only showing top 5 rows



### Adicionar uma coluna constante
Coluna com valor fixo

In [36]:
df2 = df.withColumn('country', lit('US'))
df2.show(5)

+--------------------+-------------+---+-----+---------------------+------------+-------+
|           Bank Name|         City| ST| CERT|Acquiring Institution|Closing Date|country|
+--------------------+-------------+---+-----+---------------------+------------+-------+
|The First State Bank|Barboursville| WV|14361|       MVB Bank, Inc.|    3-Apr-20|     US|
|  Ericson State Bank|      Ericson| NE|18265| Farmers and Merch...|   14-Feb-20|     US|
|City National Ban...|       Newark| NJ|21111|      Industrial Bank|    1-Nov-19|     US|
|       Resolute Bank|       Maumee| OH|58317|   Buckeye State Bank|   25-Oct-19|     US|
|Louisa Community ...|       Louisa| KY|58112| Kentucky Farmers ...|   25-Oct-19|     US|
+--------------------+-------------+---+-----+---------------------+------------+-------+
only showing top 5 rows



### Remover uma coluna

In [37]:
df2 = df.drop('CERT')
df2.show(5)

+--------------------+-------------+---+---------------------+------------+
|           Bank Name|         City| ST|Acquiring Institution|Closing Date|
+--------------------+-------------+---+---------------------+------------+
|The First State Bank|Barboursville| WV|       MVB Bank, Inc.|    3-Apr-20|
|  Ericson State Bank|      Ericson| NE| Farmers and Merch...|   14-Feb-20|
|City National Ban...|       Newark| NJ|      Industrial Bank|    1-Nov-19|
|       Resolute Bank|       Maumee| OH|   Buckeye State Bank|   25-Oct-19|
|Louisa Community ...|       Louisa| KY| Kentucky Farmers ...|   25-Oct-19|
+--------------------+-------------+---+---------------------+------------+
only showing top 5 rows



In [38]:
df2 = reduce(DataFrame.drop, ['ST', 'Address'], df)
df2.show(5)

+--------------------+-------------+-----+---------------------+------------+
|           Bank Name|         City| CERT|Acquiring Institution|Closing Date|
+--------------------+-------------+-----+---------------------+------------+
|The First State Bank|Barboursville|14361|       MVB Bank, Inc.|    3-Apr-20|
|  Ericson State Bank|      Ericson|18265| Farmers and Merch...|   14-Feb-20|
|City National Ban...|       Newark|21111|      Industrial Bank|    1-Nov-19|
|       Resolute Bank|       Maumee|58317|   Buckeye State Bank|   25-Oct-19|
|Louisa Community ...|       Louisa|58112| Kentucky Farmers ...|   25-Oct-19|
+--------------------+-------------+-----+---------------------+------------+
only showing top 5 rows



### Filter data

In [39]:
# Valores iguais
df2 = df.where(col('ST') == 'NE')

# Valores entre "Between"
df3 = df.where(df['CERT'].between('1000', '2000'))

# está dentro de varios valores "isin"

df4 = df.where(df['ST'].isin('NE', 'IL'))

print('df.count', df.count())
print('df2.count', df2.count())
print('df3.count', df3.count())
print('df4.count', df4.count())

df.count 561
df2.count 4
df3.count 9
df4.count 73


### Filtros com operadores lógicos

In [40]:
df2 = df.where((df['ST'] == 'NE') & (df['City'] == 'Ericson'))
df2.show(5)

+------------------+-------+---+-----+---------------------+------------+
|         Bank Name|   City| ST| CERT|Acquiring Institution|Closing Date|
+------------------+-------+---+-----+---------------------+------------+
|Ericson State Bank|Ericson| NE|18265| Farmers and Merch...|   14-Feb-20|
+------------------+-------+---+-----+---------------------+------------+



### Replace values in DataFrame

In [41]:
df.show(2)

print('Replace 7 in the above dataframe with 17 at all instance')
df.na.replace(7, 17).show(2)

+--------------------+-------------+---+-----+---------------------+------------+
|           Bank Name|         City| ST| CERT|Acquiring Institution|Closing Date|
+--------------------+-------------+---+-----+---------------------+------------+
|The First State Bank|Barboursville| WV|14361|       MVB Bank, Inc.|    3-Apr-20|
|  Ericson State Bank|      Ericson| NE|18265| Farmers and Merch...|   14-Feb-20|
+--------------------+-------------+---+-----+---------------------+------------+
only showing top 2 rows

Replace 7 in the above dataframe with 17 at all instance
+--------------------+-------------+---+-----+---------------------+------------+
|           Bank Name|         City| ST| CERT|Acquiring Institution|Closing Date|
+--------------------+-------------+---+-----+---------------------+------------+
|The First State Bank|Barboursville| WV|14361|       MVB Bank, Inc.|    3-Apr-20|
|  Ericson State Bank|      Ericson| NE|18265| Farmers and Merch...|   14-Feb-20|
+---------------